# Market Meanness Index MMI

## Setup

In [ ]:
!pip install finlab_crypto > log.txt
!pip install talib-binary > log.txt

In [ ]:
import finlab_crypto
finlab_crypto.setup()

In [ ]:
ohlcv = finlab_crypto.crawler.get_all_binance('BTCUSDT', '1h')

## Simple Moving Average

In [ ]:
import numpy as np
from finlab_crypto import Strategy

@Strategy(sma1=20, sma2=60)
def sma_strategy(ohlcv):
  close = ohlcv.close
  sma20 = close.rolling(sma_strategy.sma1).mean()
  sma60 = close.rolling(sma_strategy.sma2).mean()

  entries = (sma20 > sma60) & (sma20.shift() < sma60.shift())
  exits = (sma20 < sma60) & (sma20.shift() > sma60.shift())

  figures = {
      'overlaps': {
          'sma20': sma20,
          'sma60': sma60
      }
  }
  return entries, exits, figures

variables = {'sma1': np.arange(20, 100, 5), 'sma2': np.arange(20, 100, 5)}
portfolio = sma_strategy.backtest(ohlcv, variables=variables, freq='1h', plot=True)

# MMI Filter

In [ ]:
import pandas as pd

median = ohlcv.close.rolling(20).median()
p1 = ohlcv.close > median
p2 = ohlcv.close.shift() > median

# "/20" to do normalization, let values in range(0,1)
mmi = (p1 & p2).rolling(20).sum() / 20

## Plot

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

ohlcv.close['2020-10'].plot()
median['2020-10'].plot()
mmi['2020-10'].plot(secondary_y=True)

## Modify SMA strategy

In [ ]:
import numpy as np
from finlab_crypto import Strategy

@Strategy(sma1=20, sma2=60)
def mmi_sma_strategy(ohlcv):
  close = ohlcv.close
  sma20 = close.rolling(mmi_sma_strategy.sma1).mean()
  sma60 = close.rolling(mmi_sma_strategy.sma2).mean()

  entries = (sma20 > sma60) & (sma20.shift() < sma60.shift())
  exits = (sma20 < sma60) & (sma20.shift() > sma60.shift())

  figures = {
      'overlaps': {
          'sma20': sma20,
          'sma60': sma60
      }
  }

  median = ohlcv.close.rolling(20).median()
  p1 = ohlcv.close > median
  p2 = ohlcv.close.shift() > median
  mmi = (p1 & p2).astype(int).rolling(20).mean()

  # add filter condition
  entries = entries & (mmi > 0.5)

  return entries, exits, figures

variables = {'sma1': np.arange(20, 100, 5), 'sma2': np.arange(20, 100, 5)}
portfolio = mmi_sma_strategy.backtest(ohlcv, variables=variables, freq='1h', plot=True)

## Encapsulation

In [ ]:
from finlab_crypto.strategy import Filter

# must sey timeperiod default value
@Filter(timeperiod=20)
def mmi_filter(ohlcv):
  median = ohlcv.close.rolling(mmi_filter.timeperiod).median()
  p1 = ohlcv.close > median
  p2 = ohlcv.close.shift() > median
  mmi = (p1 & p2).astype(int).rolling(mmi_filter.timeperiod).mean()

  # if you don't want to show any figures, use figures={}
  figures = {
      'figures': {
          'mmi_index': mmi
      }
  }
  return mmi > 0.5, figures

f40 = mmi_filter.create({'timeperiod': 60})
f40(ohlcv)



# Result

## Optimization

In [ ]:
variables = {'sma1': np.arange(20, 100, 5), 'sma2': np.arange(20, 100, 5)}

# optimize mmi_filter timeperiod
f_mmi = mmi_filter.create({'timeperiod': np.arange(10, 50, 5)})

# add customized filter in filters parameter
portfolio = sma_strategy.backtest(ohlcv, variables=variables, filters={'mmi': f_mmi}, freq='1h', plot=True)

## Final Strategy

In [ ]:
# use the best values
variables = {'sma1': 35, 'sma2': 80}
f_mmi = mmi_filter.create({'timeperiod': 20})

portfolio = sma_strategy.backtest(ohlcv, variables=variables, filters={'mmi': f_mmi}, freq='1h', plot=True)